# Multi-Modal RAG

There are 3 ways to create multi-modal-rag
1. Use a multi-modal embedding model like CLIP or imagebind to create embedding  of images and text. REtrieve bith using similarity search and pass the documents toa multi-modal LLM
2. Use a multi-modal model to create summarues of images. Retrieve the summarues from vector storage and pass them to an llm for Q&A
3. Use a multi-modal LLM to fer description of images. Embed the text description using any embedding model of choice and store original document in doc store Retrieve summarues with a reference to the original image and text chunks. Pass the original document to a multi-modal LLM for answer generation


![image](https://av-eks-lekhak.s3.amazonaws.com/media/__sized__/article_images/mm-rag-thumbnail_webp-600x300.webp)

In [1]:
# !sudo apt install tesseract-ocr
# !sudo apt-get install poppler-utils

In [7]:
# !pip install "unstructured[all-docs]" 
!pip install  unstructured

In [7]:
# !pip install langchain==0.1.0 langchain-community==0.0.12 langchain-core==0.1.10 \
# langchain-experimental==0.0.49 \
# langchain-google-genai==0.0.6

In [3]:
from unstructured.partition.pdf import partition_pdf

img_path = "./images"
pdf_elemnts = partition_pdf(
    "./Document/llama2.pdf",
    chunking_strategy="by_title",
    extract_images_in_pdf=True,
    max_characters=3000,
    new_after_n_chars=2800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=img_path,
)

[nltk_data] Downloading package punkt to /home/darklord/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/darklord/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05_quantized.onnx:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

**Saperate text and tables**

In [4]:
from dotenv import load_dotenv
_ = load_dotenv()

In [6]:
# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables


texts, tables = categorize_elements(pdf_elemnts)

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableLambda


# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well-optimized for retrieval. Table \
    or text: {element} """
    prompt = PromptTemplate.from_template(prompt_text)
    empty_response = RunnableLambda(
        lambda x: AIMessage(content="Error processing document")
    )
    # Text summary chain
    model = VertexAI(
        temperature=0, model_name="gemini-pro", max_output_tokens=1024, to
    ).with_fallbacks([empty_response])
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 1})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries2, table_summaries = generate_text_summaries(
    texts[9:], tables, summarize_texts=True
)

In [ ]:
from langchain import PromptTemplate